In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import numpy as np
import struct
from skimage import io,data

data = []
model = []

class MnistData:
    def __init__(self, train_image_path, train_label_path,
                 test_image_path, test_label_path, augmentation):
        # file path
        self.train_image_path = train_image_path
        self.train_label_path = train_label_path
        self.test_image_path = test_image_path
        self.test_label_path = test_label_path
        self.augmentation = augmentation

        # get_data()，0:train data，1:test data
        self.train_images, self.train_labels = self.get_data(0)
        self.test_images, self.test_labels = self.get_data(1)

    def get_data(self, data_type):
        if data_type == 0:  
            image_path = self.train_image_path
            label_path = self.train_label_path
        else:  
            image_path = self.test_image_path
            label_path = self.test_label_path

        with open(image_path, 'rb') as file1:
            image_file = file1.read()
        with open(label_path, 'rb') as file2:
            label_file = file2.read()

        label_index = 0
        image_index = 0
        labels = []
        images = []
        images_rot90 = []
        images_rot180 = []
        images_rot270 = []
        images_rv1 = []
        images_rv2 = []

        # read file imformation
        magic, num_of_datasets, rows, columns = struct.unpack_from('>IIII',
                                        image_file, image_index)
        image_index += struct.calcsize('>IIII')

        for i in range(num_of_datasets):
            # read 784 unsigned byte
            temp = struct.unpack_from('>784B', image_file, image_index)
            temp = np.reshape(temp, (28, 28))
            temp = temp / 255
            images.append(temp)
            # data augmentation
            if self.augmentation == True:
              if data_type == 0:
                band_rot90 = np.rot90(temp)
                images_rot90.append(band_rot90)
                band_rot180 =  np.rot90(band_rot90)
                images_rot180.append(band_rot180)
                band_rot270 =  np.rot90(band_rot180)
                images_rot270.append(band_rot270)
                revert_1 = np.flip(temp, 0)
                images_rv1.append(revert_1)
                revert_2 = np.flip(temp, 1)
                images_rv2.append(revert_2)

            image_index += struct.calcsize('>784B')  # 每次增加784B

        # jump the describe imformation
        label_index += struct.calcsize('>II')
        labels = struct.unpack_from('>' + str(num_of_datasets) + 'B', label_file, label_index)

        # one-hot
        labels = np.eye(10)[np.array(labels)]
        # expanding data labels
        if self.augmentation == True:
          if data_type == 0:
            labels = np.concatenate((labels, labels, labels, labels, labels, labels))
            images = np.concatenate((images, images_rot90, images_rot180, images_rot270, images_rv1, images_rv2))

        return np.array(images), labels


In [ ]:
# MINIST
train_image_path = '/content/drive/My Drive/data/train-images.idx3-ubyte'
train_label_path = '/content/drive/My Drive/data/train-labels.idx1-ubyte'
test_image_path = '/content/drive/My Drive/data/t10k-images.idx3-ubyte'
test_label_path = '/content/drive/My Drive/data/t10k-labels.idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path, augmentation = False)

data.train_images = np.expand_dims(data.train_images, axis=3)
data.test_images = np.expand_dims(data.test_images, axis=3)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [24]:
# Fashion MINIST (data increament)
train_image_path = '/content/drive/My Drive/fasion_data/train-images-idx3-ubyte'
train_label_path = '/content/drive/My Drive/fasion_data/train-labels-idx1-ubyte'
test_image_path = '/content/drive/My Drive/fasion_data/t10k-images-idx3-ubyte'
test_label_path = '/content/drive/My Drive/fasion_data/t10k-labels-idx1-ubyte'

augmentation = False#@param {type:"boolean"}
data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path, augmentation)

data.train_images = np.expand_dims(data.train_images, axis=3)
data.test_images = np.expand_dims(data.test_images, axis=3)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data.train_images[2].reshape(28, 28), cmap='gray')

In [29]:
print("train.shpae test.shape", data.train_images.shape, data.test_images.shape)
print("train.shpae test.shape", data.train_labels.shape, data.test_labels.shape)

train.shpae test.shape (60000, 28, 28, 1) (10000, 28, 28, 1)
train.shpae test.shape (60000, 10) (10000, 10)


In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

# Define model
def get_model():
    # Build a sequential model
    model = tf.keras.Sequential()

    model.add(layers.InputLayer(input_shape=(28, 28, 1)))
    # Nomalization
    model.add(layers.BatchNormalization())

    # First convolutional layer
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(layers.Dropout(0.3))

    # Second convolutional layer
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(layers.Dropout(0.3))

    # Convert the output feature mapping of the previous layer into one-dimensional data
    model.add(layers.Flatten())

    # First fully connected layer
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(layers.Dropout(0.4))

    # Second fully connected layer
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dropout(0.3))

    # Nomalization
    model.add(layers.BatchNormalization())

    # Third fully connected layer
    model.add(layers.Dense(10, activation = 'softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    
    # Print model
    model.summary()

    return model

model = get_model()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 22, 22, 128)      

In [27]:
model.fit(data.train_images, data.train_labels, epochs=100, batch_size=500)

Epoch 1/100
120/120 [==============================] - 6s 40ms/step - loss: 1.2668 - accuracy: 0.5723
Epoch 2/100
120/120 [==============================] - 5s 40ms/step - loss: 0.4492 - accuracy: 0.8495
Epoch 3/100
120/120 [==============================] - 5s 40ms/step - loss: 0.3478 - accuracy: 0.8792
Epoch 4/100
120/120 [==============================] - 5s 40ms/step - loss: 0.3040 - accuracy: 0.8959
Epoch 5/100
120/120 [==============================] - 5s 40ms/step - loss: 0.2625 - accuracy: 0.9095
Epoch 6/100
120/120 [==============================] - 5s 40ms/step - loss: 0.2383 - accuracy: 0.9165
Epoch 7/100
120/120 [==============================] - 5s 40ms/step - loss: 0.2206 - accuracy: 0.9233
Epoch 8/100
120/120 [==============================] - 5s 40ms/step - loss: 0.2053 - accuracy: 0.9275
Epoch 9/100
120/120 [==============================] - 5s 40ms/step - loss: 0.1861 - accuracy: 0.9337
Epoch 10/100
120/120 [==============================] - 5s 40ms/step - loss: 0.170

In [28]:
model.evaluate(data.test_images, data.test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.3466 - accuracy: 0.9322


[0.346561998128891, 0.932200014591217]